<a href="https://colab.research.google.com/github/AbrahamKong/-CMPE257-NLP_Poem_Part_2/blob/main/NLP_Poem_Scraping_9b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Topic Modeling and Summarization for Poems
author: Abraham Kong


### Installation

In [1]:
# !pip install beautifulsoup4

In [2]:
# !pip install -U pandas-profiling

### Initialization

In [3]:
import urllib.request
import numpy as np
import pandas as pd
import re
import spacy
import bs4 as bs
from bs4 import BeautifulSoup
import json
import sys
sys.setrecursionlimit(3000)

In [4]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [5]:
def beautify_txt(text):
    # final_data = (((text).replace(u'\xa0', u' ')).replace(r'\r',u'\n').replace(r'\n', ' '))
    final_data = (((text).replace(u'\xa0', u' ')).replace(r'\r',u'\n'))

    return final_data

# Poet - Langston Hughes
# Poem - Mother To Son
Langston_Hughes_Poems = [
"https://allpoetry.com/Mother-To-Son",
"https://allpoetry.com/The-Ballad-Of-The-Landlord", 
"https://allpoetry.com/I,-Too.", 
"https://allpoetry.com/As-I-Grew-Older", 
"https://allpoetry.com/The-Negro-Mother", 
"https://allpoetry.com/The-Negro-Speaks-Of-Rivers",
"https://allpoetry.com/Trumpet-Player", 
"https://allpoetry.com/poem/8495535-Cross-by-Langston-Hughes", 
"https://allpoetry.com/Madam-and-The-Rent-Man", 
"https://allpoetry.com/Dream-Boogie"]

# Poet - Marry J Oliver
# Poem - In Blackwater woods
Mary_J_Oliver_Poems = [
"https://allpoetry.com/poem/15374219-In-Blackwater-woods-by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374221-The-Journey-by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374223-Wild-geese-by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374230-Yes--No--by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374236-Happiness-by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374239-Next-time-by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374240-The-summer-day-by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374244-Where-Does-the-Dance-Begin--Where-Does-It-End--by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374794-Breakage-by-Mary-J-Oliver", 
"https://allpoetry.com/poem/15374796-Song-for-Autumn-by-Mary-J-Oliver"]

Poems = Langston_Hughes_Poems + Mary_J_Oliver_Poems

### Scrape Data

In [6]:
poems_results=[]
def scrapePoetsPoems(urls):
  for url in urls:
    print(url)
    header = {'User-Agent':'Mozilla/5.0'}
    requests = urllib.request.Request(url,headers=header)
    requestread = urllib.request.urlopen(requests).read()
    soup = bs.BeautifulSoup(requestread,'html.parser')

    poem_content = (beautify_txt(soup.find_all('div', class_="poem_body")[0].text))
   
    poem_title = soup.find_all('h1')[0].text
    poems_results.append([poem_title, poem_content])
  return poems_results

In [7]:
import os
#------------------------------------------
# try to get the poem, if it is there, load it 
# if not scarpe it into a df. #playdefense
#
#------------------------------------------
def get_poems(poets_poems):
  file_exists = os.path.exists(poets_poems)
  print(f"{poets_poems} file_exists")
  if (file_exists):
    #load it!
    df = pd.read_csv(poets_poems, sep=';')
    print(df.head(2))
    return df
  else:
    print_the_poem(poets_poems); # call code to scrape

In [8]:
poem = scrapePoetsPoems(Poems)

https://allpoetry.com/Mother-To-Son
https://allpoetry.com/The-Ballad-Of-The-Landlord
https://allpoetry.com/I,-Too.
https://allpoetry.com/As-I-Grew-Older
https://allpoetry.com/The-Negro-Mother
https://allpoetry.com/The-Negro-Speaks-Of-Rivers
https://allpoetry.com/Trumpet-Player
https://allpoetry.com/poem/8495535-Cross-by-Langston-Hughes
https://allpoetry.com/Madam-and-The-Rent-Man
https://allpoetry.com/Dream-Boogie
https://allpoetry.com/poem/15374219-In-Blackwater-woods-by-Mary-J-Oliver
https://allpoetry.com/poem/15374221-The-Journey-by-Mary-J-Oliver
https://allpoetry.com/poem/15374223-Wild-geese-by-Mary-J-Oliver
https://allpoetry.com/poem/15374230-Yes--No--by-Mary-J-Oliver
https://allpoetry.com/poem/15374236-Happiness-by-Mary-J-Oliver
https://allpoetry.com/poem/15374239-Next-time-by-Mary-J-Oliver
https://allpoetry.com/poem/15374240-The-summer-day-by-Mary-J-Oliver
https://allpoetry.com/poem/15374244-Where-Does-the-Dance-Begin--Where-Does-It-End--by-Mary-J-Oliver
https://allpoetry.com/po

In [9]:
poems = pd.DataFrame(poem)
poems.columns = ['Title', 'Poem']

In [10]:
poems

,Title,Poem
0,Mother To Son,"\r\n\tWell, son, I'll tell you:\r\n\tLife for ..."
1,The Ballad Of The Landlord,"Landlord, landlord,\nMy roof has sprung a leak..."
2,"I, Too.","\r\n\tI, too, sing America.\r\n\tI am the dark..."
3,As I Grew Older,It was a long time ago.\nI have almost forgott...
4,The Negro Mother,"\r\n\tChildren, I come back today\r\n\tTo tell..."
5,The Negro Speaks Of Rivers,I've known rivers:I've known rivers ancient as...
6,Trumpet Player,The Negro\nwith the trumpet at his lips\nhas d...
7,Cross,My old man's a white old manAnd my old mother'...
8,Madam and The Rent Man,"The rent man knocked. \nHe said, Howdy-do? \nI..."
9,Dream Boogie,"\r\n\tGood morning, daddy!\r\n\tAin't you hear..."


### NLP Pre-Processing

#### Installation

In [11]:
# # install spacy large english language model to use
# ! python -m spacy download en_core_web_lg

In [12]:
# exit()

#### POS

In [13]:
nlp = spacy.load('en_core_web_sm')

In [14]:
corpus1 = poems['Poem'][0]
corpus2 = poems['Poem'][1]

In [15]:
print("Corpus of poem 1:\n")
print(corpus1)
print("\nCorpus of poem 2:\n")
print(corpus2)

Corpus of poem 1:


	Well, son, I'll tell you:
	Life for me ain't been no crystal stair.
	It's had tacks in it,
	And splinters,
	And boards torn up,
	And places with no carpet on the floor—
	Bare.
	But all the time
	I'se been a-climbin' on,
	And reachin' landin's,
	And turnin' corners,
	And sometimes goin' in the dark
	Where there ain't been no light.
	So, boy, don't you turn back.
	Don't you set down on the steps.
	'Cause you finds it's kinder hard.
	Don't you fall now—
	For I'se still goin', honey,
	I'se still climbin',
	And life for me ain't been no crystal stair. © by owner. provided at no charge for educational purposes    

Corpus of poem 2:

Landlord, landlord,
My roof has sprung a leak.
Don't you 'member I told you about it
Way last week?

Landlord, landlord,
These steps is broken down.
When you come up yourself
It's a wonder you don't fall down.

Ten Bucks you say I owe you?
Ten Bucks you say is due?
Well, that's Ten Bucks more'n I'l pay you
Till you fix this house up new.

Wh

In [16]:
removed_char_list = [[]] * poems.shape[0]

In [17]:
for i in range(poems.shape[0]):
  removed_char_list[i] = [re.sub(r"[^a-zA-Z0-9]+", ' ', doc) for doc in poems['Poem'][i]]

In [18]:
for i in range(poems.shape[0]):
  removed_char_list[i] = ''.join([str(elem) for elem in removed_char_list[i]])

In [19]:
for i in range(poems.shape[0]):
  print("Clean Version of poem ")
  print(i) 
  print(" ")
  print(removed_char_list[i])

Clean Version of poem 
0
 
   Well  son  I ll tell you    Life for me ain t been no crystal stair    It s had tacks in it    And splinters    And boards torn up    And places with no carpet on the floor    Bare    But all the time   I se been a climbin  on    And reachin  landin s    And turnin  corners    And sometimes goin  in the dark   Where there ain t been no light    So  boy  don t you turn back    Don t you set down on the steps     Cause you finds it s kinder hard    Don t you fall now    For I se still goin   honey    I se still climbin     And life for me ain t been no crystal stair    by owner  provided at no charge for educational purposes    
Clean Version of poem 
1
 
Landlord  landlord  My roof has sprung a leak  Don t you  member I told you about it Way last week   Landlord  landlord  These steps is broken down  When you come up yourself It s a wonder you don t fall down   Ten Bucks you say I owe you  Ten Bucks you say is due  Well  that s Ten Bucks more n I l pay you 

In [20]:
poems['Doc'] = np.NaN

In [21]:
for i in range(poems.shape[0]):
  poems['Doc'][i] = nlp(removed_char_list[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [22]:
for i in range(poems.shape[0]):
  print("POS poem ")
  print(i) 
  print(" ")
  print(poems['Doc'][i])

POS poem 
0
 
   Well  son  I ll tell you    Life for me ain t been no crystal stair    It s had tacks in it    And splinters    And boards torn up    And places with no carpet on the floor    Bare    But all the time   I se been a climbin  on    And reachin  landin s    And turnin  corners    And sometimes goin  in the dark   Where there ain t been no light    So  boy  don t you turn back    Don t you set down on the steps     Cause you finds it s kinder hard    Don t you fall now    For I se still goin   honey    I se still climbin     And life for me ain t been no crystal stair    by owner  provided at no charge for educational purposes    
POS poem 
1
 
Landlord  landlord  My roof has sprung a leak  Don t you  member I told you about it Way last week   Landlord  landlord  These steps is broken down  When you come up yourself It s a wonder you don t fall down   Ten Bucks you say I owe you  Ten Bucks you say is due  Well  that s Ten Bucks more n I l pay you Till you fix this house up

In [23]:
pos_list = [[]] * poems.shape[0]
poems['POS'] = np.empty((len(poems), 0)).tolist()

In [24]:
for i in range(poems.shape[0]):
  for token in poems['Doc'][i]:
    if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
      poems['POS'][i].append(token.text)


poems['POS']

0     [son, ll, tell, Life, ain, t, crystal, stair, ...
1     [Landlord, landlord, roof, sprung, leak, Don, ...
2     [sing, America, darker, brother, send, eat, ki...
3     [long, time, forgotten, dream, front, Bright, ...
4     [Children, come, today, tell, story, long, dar...
5     [ve, known, rivers, ve, known, rivers, ancient...
6     [Negro, trumpet, lips, dark, moons, weariness,...
7     [old, man, s, white, old, manAnd, old, mother,...
8     [rent, man, knocked, said, said, Can, said, kn...
9     [Good, morning, daddy, Ain, t, heard, boogie, ...
10    [Look, trees, turning, own, bodies, pillars, l...
11    [One, day, knew, began, voices, kept, shouting...
12    [good, walk, knees, hundred, miles, desert, re...
13    [necessary, opinions, think, spotted, trout, l...
14    [afternoon, watched, bear, looking, secret, bi...
15    [Next, time, d, look, earth, saying, d, stop, ...
16    [made, world, made, swan, black, bear, made, g...
17    [Don, t, call, world, adorable, useful, s,

In [25]:
poems

,Title,Poem,Doc,POS
0,Mother To Son,"\r\n\tWell, son, I'll tell you:\r\n\tLife for ...","( , Well, , son, , I, ll, tell, you, , ...","[son, ll, tell, Life, ain, t, crystal, stair, ..."
1,The Ballad Of The Landlord,"Landlord, landlord,\nMy roof has sprung a leak...","(Landlord, , landlord, , My, roof, has, spru...","[Landlord, landlord, roof, sprung, leak, Don, ..."
2,"I, Too.","\r\n\tI, too, sing America.\r\n\tI am the dark...","( , I, , too, , sing, America, , I, am,...","[sing, America, darker, brother, send, eat, ki..."
3,As I Grew Older,It was a long time ago.\nI have almost forgott...,"(It, was, a, long, time, ago, , I, have, almo...","[long, time, forgotten, dream, front, Bright, ..."
4,The Negro Mother,"\r\n\tChildren, I come back today\r\n\tTo tell...","( , Children, , I, come, back, today, , T...","[Children, come, today, tell, story, long, dar..."
5,The Negro Speaks Of Rivers,I've known rivers:I've known rivers ancient as...,"(I, ve, known, rivers, I, ve, known, rivers, a...","[ve, known, rivers, ve, known, rivers, ancient..."
6,Trumpet Player,The Negro\nwith the trumpet at his lips\nhas d...,"(The, Negro, with, the, trumpet, at, his, lips...","[Negro, trumpet, lips, dark, moons, weariness,..."
7,Cross,My old man's a white old manAnd my old mother'...,"(My, old, man, s, a, white, old, manAnd, my, o...","[old, man, s, white, old, manAnd, old, mother,..."
8,Madam and The Rent Man,"The rent man knocked. \nHe said, Howdy-do? \nI...","(The, rent, man, knocked, , He, said, , How...","[rent, man, knocked, said, said, Can, said, kn..."
9,Dream Boogie,"\r\n\tGood morning, daddy!\r\n\tAin't you hear...","( , Good, morning, , daddy, , Ain, t, yo...","[Good, morning, daddy, Ain, t, heard, boogie, ..."


### Similarity between Poet 1 (Langston Hughes) and Poet 2 (Mary J Oliver)

In [26]:
poet1= poems['Doc'][0]
poet2= poems['Doc'][10]

print('Similariy score between the two poets\n')
print(poet1.similarity(poet2))

Similariy score between the two poets

0.7823405474316169


/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


### Save to JSON

In [27]:
poems.iloc[0].to_json(r'/content/poet1.json')
poems.iloc[1].to_json(r'/content/poet2.json')

OverflowError: ignored

## Create new poem

In [ ]:
new_poem1 = str(corpus1)

In [ ]:
doc1_mappings = {}

In [ ]:
for x in poem1_pos:
  max=-1
  for y in poem2_pos:
    score=nlp(x).similarity(nlp(y))
    if score > max:
    doc1_mappings[x]=[y,score]
    max=score

In [ ]:
for key, value in doc1_mappings.items():
  # Replace key POS with value POS in string
  new_poem1 = new_poem1.replace(str(key), str(value[0]))

In [ ]:
new_poem1 = beautify_txt(new_poem1)
new_poem1

## Transformers

### Functions

In [ ]:
def cleanPoemText(text):
    
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'re", " are", text)

    if embedding is not 'BERT':
        text = re.sub(r"[0-9]+", ' ', text)
        text = re.sub(r"-", ' ', text)
    
    
    text = text.strip().lower()
    
    if embedding is 'WORD2VEC_NO_STOP':
        # Removal of Stop words
        default_stop_words = set(stopwords.words('english'))
        default_stop_words.difference_update({'no', 'not', 'nor', 'too', 'any'})
        stop_words = default_stop_words.union({"'m", "n't", "'d", "'re", "'s",
                                               'would','must',"'ve","'ll",'may'})
    
        word_list = word_tokenize(text)
        filtered_list = [w for w in word_list if not w in stop_words]
        text = ' '.join(filtered_list)
    
    if embedding is not 'BERT':
        # Removal of other contractions
        text = re.sub(r"'", ' ', text)
    
    # Replace punctuations with space
    if embedding is 'BERT': # save ! ? . for end of the sentence detection [,/():;']
        filters='"#$%&*+<=>@[\\]^_`{|}~\t\n'
        text = re.sub(r'\!+', '!', text)
        text = re.sub(r'\?+', '?', text)
    else:
        filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    if embedding is 'BERT':
        text = re.sub(r'\( *\)', ' ', text)

    if embedding is not 'BERT':
        text = ' '.join([w for w in text.split() if len(w)>1])

    # Replace multiple space with one space
    text = re.sub(' +', ' ', text)
    
    text = ''.join(text)

    return text

In [ ]:
def normalization_pos(text):
    # Lemmatization and Stemming according to POS tags

    word_list = word_tokenize(text)
    rev = []
    lemmatizer = WordNetLemmatizer() 
    stemmer = PorterStemmer() 
    for word, tag in pos_tag(word_list):
        if tag.startswith('J'):
            w = lemmatizer.lemmatize(word, pos='a')
        elif tag.startswith('V'):
            w = lemmatizer.lemmatize(word, pos='v')
        elif tag.startswith('N'):
            w = lemmatizer.lemmatize(word, pos='n')
        elif tag.startswith('R'):
            w = lemmatizer.lemmatize(word, pos='r')
        else:
            w = word
        w = stemmer.stem(w)
        rev.append(w)
    review = ' '.join(rev)
    return review

## Embedding

### Installation

In [ ]:
# !pip install bert-embedding
# !pip install sentence-transformers

### Initialization

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.tag import pos_tag
import re
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from bert_embedding import BertEmbedding
from sentence_transformers import SentenceTransformer
import gensim 
from gensim.models import Word2Vec 
#Glove, COVE, FastText and other langauge embeddings 

In [ ]:
embedding = ''
# for Word2Vec with stop words
poems['Poem'] = poems['Poem'].apply(cleanPoemText)
poems.head(1)

In [ ]:
poems['clean_text_normalized'] = poems['Poem'].apply(normalization_pos)

In [ ]:
poems